In [1]:
# This cell loads in the API
# and then prints out the serial ports

# !pip install --upgrade git+git://github.com/OpenTrons/opentrons_sdk.git@master#egg=opentrons

from opentrons import containers
from opentrons import instruments
from opentrons.robot import Robot

robot = Robot.get_instance()
# robot.get_serial_ports_list()

In [2]:
# this cell connects to a robot and immediately homes
# if .connect() is called without a port, the smoothieboard is simulated

robot.connect() # virtual smoothieboard
# robot.connect('/dev/tty.usbmodem1421')
# robot.home(now=True)


True

In [3]:
# this cells is similar to the Deck and Head sections in a JSON protocol

# Create a JSON protocol with the exact same containers and pipettes as here
# They must be the same type, have the same user-defined names, and pipette's on the same axis (a or b)

p10rack = containers.load(
    'tiprack-10ul',  # container type
    'C2',             # slot
    'p10-rack'         # user-defined name, optional for now
)
p200rack = containers.load(
    'tiprack-200ul',  # container type
    'A1',             # slot
    'p200-rack'         # user-defined name, optional for now
)
trough = containers.load(
    'trough-12row',
    'E1',
    'trough'
)
plate = containers.load(
    '96-PCR-flat',
    'C1',
    'plate'
)
trash = containers.load(
    'point',
    'B2',
    'trash'
)
    
p200 = instruments.Pipette(
    name="p200", # optional
    trash_container=trash,
    tip_racks=[p200rack],
    min_volume=20, # actual minimum volume of the pipette
    axis="a",
    channels=8 # 
)

p200.set_max_volume(200)  # volume calibration, can be called whenever you want

p10 = instruments.Pipette(
    name="p10", # optional
    trash_container=trash,
    tip_racks=[p10rack],
    min_volume=1, # actual minimum volume of the pipette
    axis="b",
    channels=1 # 1 o
)
    
p10.set_max_volume(10)

In [4]:
# assume these
sample_numbers = 3
sample_location = plate.rows[0] #first row of plate
diluent_location = trough['A1'] #what the samples are diluted with

# inputs from user
dilution_number = 3 # how many dilutions
dilution_factor = 10 # dilution factor 10X
dilution_volume = 200 # in uL

# calculate remaining volumes
# C1V1 = C2V2
# (initial concentration)(initial volume) = (final concentration)(final volume)
# (dilution_factor)(sample_volume) = (1X)(dilution_volume)

sample_volume = (dilution_volume)/(dilution_factor)
diluent_volume = dilution_volume - sample_volume

In [5]:
# distribute diluent_volume
robot.clear()

# add diluent to plate based on volume
if diluent_volume <= 10:    
    p200.pick_up_tip()
    for i in range(1, dilution_number + 1):
        p200.aspirate(diluent_volume, trough['A1']).dispense(plate.rows[i])
    p200.drop_tip()
    
    for col in range(sample_numbers):
        p10.pick_up_tip()
        for row in range(1, dilution_number):
            p10.aspirate(diluent_volume, plate.rows[row][col]).dispense(plate.rows[row + 1][col])
        p10.drop_tip()
        
else:
    for i in range(1, dilution_number + 1):
        p200.aspirate(diluent_volume, trough['A1']).dispense(plate.rows[i])
    p200.drop_tip()

# dilute samples through wells based on volume    
if sample_volume <= 10:
    for col in range(sample_numbers):
        p10.pick_up_tip()
        for row in range(0, dilution_number):
            p10.aspirate(sample_volume, plate.rows[row][col]).dispense(plate.rows[row + 1][col]).mix(sample_volume, 3, plate.rows[row + 1][col])
        p10.drop_tip() 
else:
    # use p200
    p200.pick_up_tip()
    for i in range(0, dilution_number):
        p200.aspirate(sample_volume, plate.rows[i]).dispense(plate.rows[i + 1]).mix(sample_volume, 3, plate.rows[i + 1])
    p200.drop_tip()

In [6]:
print('\n'.join([str(c.description) for c in robot._commands]))
robot.simulate()

Aspirating 180.0uL at <Deck>/<Slot E1>/<Container trough>/<Well A1>
Dispensing <Series: <Well A2> <Well B2> <Well C2> <Well D2> <Well E2> <Well F2> <Well G2> <Well H2>>uL at <In Place>
Aspirating 180.0uL at <Deck>/<Slot E1>/<Container trough>/<Well A1>
Dispensing <Series: <Well A3> <Well B3> <Well C3> <Well D3> <Well E3> <Well F3> <Well G3> <Well H3>>uL at <In Place>
Aspirating 180.0uL at <Deck>/<Slot E1>/<Container trough>/<Well A1>
Dispensing <Series: <Well A4> <Well B4> <Well C4> <Well D4> <Well E4> <Well F4> <Well G4> <Well H4>>uL at <In Place>
Drop_tip at <In Place>
Picking up tip from <In Place>
Aspirating 20.0uL at <Deck>/<Slot C1>/<Container plate>/<Series: <Well A1> <Well B1> <Well C1> <Well D1> <Well E1> <Well F1> <Well G1> <Well H1>>
Dispensing <Series: <Well A2> <Well B2> <Well C2> <Well D2> <Well E2> <Well F2> <Well G2> <Well H2>>uL at <In Place>
Mixing 3 times with a volume of 0ul
Aspirating 20.0uL at <Deck>/<Slot C1>/<Container plate>/<Series: <Well A2> <Well B2> <Well C

["0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul).",
 "0.0µl is less than pipette's min_volume (20ul)."]

In [7]:
robot.connect(robot.get_serial_ports_list()[0])
robot.run()

RuntimeWarning: Y limit switch hit

In [8]:
robot.home(now=True)

True